In [81]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
#This could be moved into a .py file and imported in both notebooks

import tensorflow as tf
from tensorflow.keras import layers

class MyNER(tf.keras.Model):
    def __init__(self, max_seq_len, embed_input_dim, embed_output_dim, embed_weights, num_labels):
        super(MyNER, self).__init__() 
        self.embedding = layers.Embedding(input_dim=embed_input_dim, 
        output_dim=embed_output_dim, weights=embed_weights,    
        input_length=max_seq_len, trainable=False, mask_zero=True)

        self.bilstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True))
        self.dense = layers.Dense(num_labels)
        
    def call(self, inputs):
        x = self.embedding(inputs) # batchsize, max_seq_len, embedding_output_dim
        
        x = self.bilstm(x) # batchsize, max_seq_len, hidden_dim_bilstm
        
        logits = self.dense(x) #batchsize, max_seq_len, num_labels
        
        return logits

In [39]:
import pickle

with open('bilstm_model_0/settings.pkl', 'rb') as f_in:
    settings = pickle.load(f_in)
    
with open('bilstm_model_0/mappings.pkl', 'rb') as f_in:
    mappings = pickle.load(f_in)

In [7]:
test_model = MyNER(**settings)
test_model.load_weights(f"bilstm_model_0/weights")

In [8]:
import spacy


In [112]:
class InferenceModel():
    def __init__(self, model, mappings, settings):
        self.model = model
        self.nlp = spacy.load('en_core_web_sm')
        self.settings = settings
        self.mappings = mappings
        self.word2id = self.mappings['word2id']
        self.id2label = self.mappings['id2label']
        self.batch_size = 2 # this may be different from the one used at training, depending on the server resources
        
    def create_features(self, word_sentences):
        id_sentences = []
        for word_sentence in word_sentences:
            word_indices = []
            for word in word_sentence:
                if word.lower() in self.word2id:
                    word_idx = self.word2id[word.lower()]
                else:
                    word_idx = self.word2id['UnkWord']
                word_indices.append(word_idx)
            id_sentences.append(word_indices)
        
        padded_sentences = pad_sequences(id_sentences, self.settings['max_seq_len'], padding='post')
        return padded_sentences
    
    def indices_to_labels(self, indices):
        labels = []
        for batch in indices:
            for sentence in batch:
                label = [self.id2label[idx] for idx in sentence]
                labels.append(label)
        return labels
    
    def format_response(self, row):
        response = []
        for i in range(len(row.words)):
            if row.pred_labels[i] != 'B-INGREDIENT':
                continue
            start = row.tokens[i].idx
            j = i + 1
            while j < len(row.words) and row.pred_labels[j] != 'O':
                j += 1
            # j-1 is the last token from the current INGREDIENT
            end = row.tokens[j-1].idx + len(row.tokens[j-1].text)
            ingredient = row.Directions[start:end]
            response.append([ingredient, start, end])
        return response
        
    def predict(self, request):
        df_request = pd.DataFrame(request.items(), columns=['Recipe Name', 'Directions'])
        df_request['texts'] = df_request.Directions.map(lambda x: x if pd.notna(x) else '')
        df_request['docs'] = df_request.texts.map(lambda text: self.nlp(text))
        df_request['tokens'] = df_request.docs.map(lambda doc: [token for token in doc])
        df_request['words'] = df_request.docs.map(lambda doc: [token.text for token in doc])
        
        features = self.create_features(df_request.words)
        
        dataset = tf.data.Dataset.from_tensor_slices(features)
        dataset = dataset.batch(self.batch_size, drop_remainder=False)
        
        pred_labels = []
        for sentences_batch in dataset:
            logits = self.model(sentences_batch)
            probs = tf.nn.softmax(logits)
            preds = tf.argmax(probs, axis=2)
            pred_labels.append(np.asarray(preds))
        
        df_request['pred_labels'] = self.indices_to_labels(pred_labels)
        df_request['response'] = df_request.apply(self.format_response, axis=1)
        
        response_dict = {}
        for _, row in df_request.iterrows():
            response_dict[row['Recipe Name']] = row.response
        
        return response_dict

In [118]:

req = {
    "recipe1": "This is recipe 1 owdquyvgk",
    "recipe2": "This is recipe 2 directions",
    "Cream of Cauliflower Soup II Recipe": 'In a large pot over medium heat, melt butter.  Stir in onion and garlic and cook until onion is translucent, about 5 minutes.  Stir in potatoes and carrots and cook 5 minutes more.  Pour in chicken broth and bring to a boil.  Stir in cauliflower, cover, reduce heat and simmer until vegetables are tender, 10 to 20 minutes.  Remove from heat.**Puree in batches in a blender or food processor, or in the pot using an immersion blender.  Return to low heat and stir in milk, salt, pepper, nutmeg and sherry.  Heat through.  Serve garnished with parsley.**',
    
}



In [114]:
inference_model = InferenceModel(test_model, mappings, settings)

In [115]:
res = inference_model.predict(req)

In [117]:
res

{'recipe1': [],
 'recipe2': [],
 'Cream of Cauliflower Soup II Recipe': [['butter', 38, 44],
  ['onion', 55, 60],
  ['garlic', 65, 71],
  ['onion', 87, 92],
  ['potatoes', 135, 143],
  ['carrots', 148, 155],
  ['chicken broth', 190, 203],
  ['cauliflower', 234, 245],
  ['milk', 467, 471],
  ['salt', 473, 477],
  ['nutmeg', 487, 493],
  ['sherry', 498, 504],
  ['parsley', 543, 550]]}